In [207]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA

In [208]:
# import demographic info from all datasets

enigma=pd.read_csv('/Users/emilyolafson/GIT/ENIGMA/data/Behaviour_Information_ALL_April7_2022.csv', header=0)
hcpaging= pd.read_csv('/Users/emilyolafson/GIT/ENIGMA/data/HCPAGING/hcpaging_subject_info.csv', header=0)
hcpya = pd.read_csv('/Users/emilyolafson/GIT/ENIGMA/data/HCP_YA/hcp_subjinfo_age_sex.csv', header=0)

enigma.rename(columns={'BIDS_ID':'Subject', 'SEX':'sex', 'AGE':'age'},inplace=True)
hcpaging.rename(columns={'subjectid':'Subject'},inplace=True)
hcpya.rename(columns={'subjectid':'Subject'},inplace=True)

enigma_id = enigma.pop('Subject')
enigma_age = enigma.pop('age')
enigma_sex = enigma.pop('sex')

hcpaging_id = hcpaging.pop('Subject')
hcpaging_age = hcpaging.pop('age')
hcpaging_sex = hcpaging.pop('sex')

hcpya_id = hcpya.pop('Subject')
hcpya_age = hcpya.pop('age')
hcpya_sex = hcpya.pop('sex')
enigma_agesex=pd.DataFrame(data=np.transpose([enigma_id, enigma_age, enigma_sex]), columns=['Subject', 'age', 'sex'])
hcpaging_agesex=pd.DataFrame(data=np.transpose([hcpaging_id, hcpaging_age, hcpaging_sex]), columns=['Subject', 'age', 'sex'])
hcpya_agesex=pd.DataFrame(data=np.transpose([hcpya_id, hcpya_age, hcpya_sex]), columns=['Subject', 'age', 'sex'])

# import CT from all datasets

# ENIGMA
# load cortical thickness data to get headers (regions)
enigma_rh_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/aseg_aparc/aparc_rh_stats_thickness.txt', header=0)
enigma_lh_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/aseg_aparc/aparc_lh_stats_thickness.txt', header=0)
enigma_rh_thickness.rename(columns = {'rh.aparc.thickness':'Subject'},inplace=True)
enigma_lh_thickness.rename(columns = {'lh.aparc.thickness':'Subject'},inplace=True)

enigma_thickness = enigma_lh_thickness.merge(enigma_rh_thickness, on='Subject')

ids = enigma_thickness['Subject']
lst = [e[5:] for e in ids]
site = [e[0:4] for e in ids]
enigma_thickness['Subject']=lst

# HCPAGING
hcpaging_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/HCPAGING/hcpaging_aparc_thickness_stats.tsv', header=0)
ids = hcpaging_thickness['Subject']
lst = [e[:10] for e in ids]
hcpaging_thickness['Subject']=lst

# HCP YA
hcpya_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/HCP_YA/hcp_aparc_thickness_stats.tsv', header=0)

# merge age and sex columns together for plotting 
enigma_all = enigma_thickness.merge(enigma_agesex, on='Subject')
enigma_all['dataset']='ENIGMA'
enigma_all['site']=site
enigma_all=enigma_all.replace({'sex': {1: 'F', 2: 'M'}})
hcpaging_all = hcpaging_thickness.merge(hcpaging_agesex, on='Subject')
hcpaging_all['dataset']='HCP_AGING'
hcpaging_all['site']='HCP_AGING'
hcpya_all = hcpya_thickness.merge(hcpya_agesex, on='Subject')
hcpya_all['dataset']='HCP_YA'
hcpya_all['site']='HCP_YA'

# merge hcpaging and hcpya
hcp_controls=pd.concat((hcpaging_all,hcpya_all), axis=0)

hcp_controls.to_csv('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/hcp_aging_ya_thickness_datasetvar.csv')
hcp_controls['disease_status']=0
enigma_all['disease_status']=1

thickness_all = pd.concat((hcp_controls, enigma_all), axis=0)

thickness_all.to_csv('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/hcp_aging_ya_enigma_thickness_datasetvar.csv')

np.where(thickness_all['sex']==3)

thickness_all.iloc[2401]
# COMBAT

Subject                                 sub-r046s008
lh_bankssts_thickness                          2.557
lh_caudalanteriorcingulate_thickness            2.98
lh_caudalmiddlefrontal_thickness               2.641
lh_cuneus_thickness                            1.843
                                            ...     
age                                             51.0
sex                                              3.0
dataset                                       ENIGMA
site                                            R046
disease_status                                     1
Name: 679, Length: 76, dtype: object

In [209]:
combat_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/hcp_aging_ya_thickness_datasetvar_COMBAT.csv', header=0, sep=',')


In [210]:
# get the mean and std for each age using a windowed approach

size_groups = np.empty(shape=(73,1))
ages = np.empty(shape=(73,1))
mean_groups = np.empty(shape=(73,len(combat_thickness.columns[1:71])))
std_groups = np.empty(shape=(73,len(combat_thickness.columns[1:71])))
prop_hcpaging= np.empty(shape=(73,1))
prop_hcpya= np.empty(shape=(73,1))
size_nonwindow= np.empty(shape=(73,1))

counter=0
for age in range(21+1,93+2):
    print(age-1)
    print(age+1)
    print('age:')
    print('new')
    subset=combat_thickness[(combat_thickness['age'] >= age-1) & (combat_thickness['age'] <= age+1)]
    print(subset['age'])
    size_groups[counter]=subset.shape[0]
    mean_groups[counter,:]=np.mean(subset.iloc[:,1:71],axis=0)
    std_groups[counter,:]=np.std(subset.iloc[:,1:71],axis=0)
    prop_hcpaging[counter,:]=np.sum(subset['dataset']=='HCP_AGING')
    prop_hcpya[counter,:]=np.sum(subset['dataset']=='HCP_YA')
    size_nonwindow[counter,:]=np.sum((np.round(combat_thickness['age']) == age))

    ages[counter]=age
    counter=counter+1



21
23
age:
new
730     22.0
737     23.0
757     22.0
760     22.0
762     22.0
        ... 
1688    22.0
1696    23.0
1704    23.0
1706    22.0
1715    22.0
Name: age, Length: 109, dtype: float64
22
24
age:
new
730     22.0
733     24.0
737     23.0
757     22.0
760     22.0
        ... 
1696    23.0
1704    23.0
1706    22.0
1712    24.0
1715    22.0
Name: age, Length: 155, dtype: float64
23
25
age:
new
733     24.0
737     23.0
740     25.0
743     25.0
759     25.0
        ... 
1651    25.0
1687    25.0
1696    23.0
1704    23.0
1712    24.0
Name: age, Length: 167, dtype: float64
24
26
age:
new
733     24.0
735     26.0
738     26.0
740     25.0
743     25.0
        ... 
1661    26.0
1668    26.0
1687    25.0
1701    26.0
1712    24.0
Name: age, Length: 190, dtype: float64
25
27
age:
new
725     27.0
726     27.0
728     27.0
734     27.0
735     26.0
        ... 
1693    27.0
1697    27.0
1701    26.0
1707    27.0
1713    27.0
Name: age, Length: 244, dtype: float64
26
28
age:
new


In [211]:
cols = combat_thickness.columns[1:71]
meancols = [col+'_mean' for col in cols]
stdcols = [col+'_std' for col in cols]

colnames_means = ['age', 'size', 'prop_aging', 'prop_ya', 'sizenowindow'] + meancols 
colnames_stds = ['age', 'size', 'prop_aging', 'prop_ya','sizenowindow'] + stdcols 

In [212]:
tmp_m= np.concatenate([ages,size_groups,prop_hcpaging, prop_hcpya,size_nonwindow, mean_groups],axis=1)
tmp_s= np.concatenate([ages,size_groups,prop_hcpaging, prop_hcpya, size_nonwindow,std_groups],axis=1)

meantable = pd.DataFrame(data=tmp_m, columns=colnames_means)
stdtable = pd.DataFrame(data=tmp_s, columns=colnames_stds)

In [ ]:
enigma_combat_thickness =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER_ENIGMA/enigma_thickness_datasetvar_COMBAT.csv', header=0, sep=',')


In [219]:
fig, axs = plt.subplots(ncols=5, nrows=7,figsize=(25,35))
counter =0
rowcount=0
# left hem

for i in range(5,40):
    yvar= meantable.columns[i]
    y_std = stdtable.iloc[:,i]
    sns.lineplot(x='age', y=yvar, data=meantable, ax=axs[rowcount,counter%5],palette="hls")
    axs[rowcount,counter%5].fill_between(meantable['age'], meantable[yvar] - y_std,  meantable[yvar] + y_std, color='b', alpha=0.2)
    axs[rowcount,counter%5].scatter(x=enigma_combat_thickness['age'], y=enigma_combat_thickness.iloc[:,i-4],s=1,color='#ba2c4a')

    axs[rowcount,counter%5].set_ylim([1.2, 5])
    axs[rowcount,counter%5].set_ylim([1.2, 5])
    axs[rowcount,counter%5].tick_params(axis='x', labelsize=10)
    axs[rowcount,counter%5].tick_params(axis='y', labelsize=10)
    axs[rowcount,counter%5].legend(['ComBat-adjusted HCP mean',' +/- 1 standard deviation','ComBat-Adjusted ENIGMA'])

   # axs[rowcount,counter%5].get_legend().remove()
    counter = counter +1
    if (counter%5 ==0) & (counter!=0):
        rowcount = rowcount+1

plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/lefthem_CT_hcp_YA_AGING_meanstd.jpeg',bbox_inches='tight')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/lefthem_CT_hcp_YA_AGING_meanstd.pdf',bbox_inches='tight')

In [218]:
fig, axs = plt.subplots(ncols=5, nrows=7,figsize=(25,35))
counter =0
rowcount=0
# left hem

for i in range(40,75):
    yvar= meantable.columns[i]
    y_std = stdtable.iloc[:,i]
    sns.lineplot(x='age', y=yvar, data=meantable, ax=axs[rowcount,counter%5],palette="hls")
    axs[rowcount,counter%5].fill_between(meantable['age'], meantable[yvar] - y_std,  meantable[yvar] + y_std, color='b', alpha=0.2)
    axs[rowcount,counter%5].scatter(x=enigma_combat_thickness['age'], y=enigma_combat_thickness.iloc[:,i-4],s=1,color='#ba2c4a')

    axs[rowcount,counter%5].set_ylim([1.2, 5])
    axs[rowcount,counter%5].set_ylim([1.2, 5])
    axs[rowcount,counter%5].tick_params(axis='x', labelsize=10)
    axs[rowcount,counter%5].tick_params(axis='y', labelsize=10)
    axs[rowcount,counter%5].legend(['ComBat-adjusted HCP mean',' +/- 1 standard deviation','ComBat-Adjusted ENIGMA'])

   # axs[rowcount,counter%5].get_legend().remove()
   # axs[rowcount,counter%5].legend(fontsize=8)
    counter = counter +1
    if (counter%5 ==0) & (counter!=0):
        rowcount = rowcount+1

plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/righthem_CT_hcp_YA_AGING_meanstd.jpeg',bbox_inches='tight')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/righthem_CT_hcp_YA_AGING_meanstd.pdf',bbox_inches='tight')

In [149]:
plt.figure(figsize=(8,4))

plt.plot(meantable['age'], meantable['sizenowindow'],'-o',color='#6b337b')
plt.xlabel('Age')
plt.ylabel('Number of subjects')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/age_hcp_sizes.pdf')


plt.figure(figsize=(8,4))

plt.plot(meantable['age'], meantable['sizenowindow'],'-o',color='#6b337b')
plt.plot(meantable['age'], meantable['prop_aging'],'-o',color='#e42021')
plt.plot(meantable['age'], meantable['prop_ya'],'-o',color='#397eb8')

plt.ylim(0,100)
plt.xlabel('Age')
plt.ylabel('Number of subjects')
plt.legend(['All HCP - No windowing', 'HCP Aging - 3-Year Windowing','HCP Young Adult - 3-Year Windowing'])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/age_hcp_sizes_hcpaging.pdf',bbox_inches='tight')



In [150]:
plt.figure(figsize=(8,4))
plt.plot(np.int8(meantable['age']), meantable['prop_aging'],'-o',color='#e42021')
plt.plot(np.int8(meantable['age']), meantable['prop_ya'],'-o',color='#397eb8')
plt.hist(enigma_combat_thickness['age'], bins=range(20, 100, 3),color='#cfc428',alpha=0.5)
plt.xlabel('Age')
plt.ylabel('Number of subjects in 3-year window')
plt.legend(['HCP Aging', 'HCP Young Adult', 'ENIGMA subjects age'])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/age_windows_sizes.pdf',bbox_inches='tight')


In [193]:
i=69
yvar= meantable.columns[i]

print(yvar)
enigma_combat_thickness.iloc[:,i-4]

rh_supramarginal_thickness_mean


KeyError: 65

In [205]:
fig, axs = plt.subplots(ncols=2, nrows=1,figsize=(10,5))

i=34
yvar= meantable.columns[i]
y_std = stdtable.iloc[:,i]
sns.lineplot(x='age', y=yvar, data=meantable, ax=axs[0],palette="hls")
axs[0].fill_between(meantable['age'], meantable[yvar] - y_std,  meantable[yvar] + y_std, color='b', alpha=0.2)

axs[0].scatter(x=enigma_combat_thickness['age'], y=enigma_combat_thickness.iloc[:,i-4],s=1,color='#ba2c4a')
axs[0].set_ylim([1.8, 4])
axs[0].set_ylim([1.8, 4])
axs[0].tick_params(axis='x', labelsize=10)
axs[0].tick_params(axis='y', labelsize=10)
axs[0].legend(['ComBat-adjusted HCP mean',' +/- 1 standard deviation','ComBat-Adjusted ENIGMA'])

i=52
sns.lineplot(x='age', y=yvar, data=meantable, ax=axs[1],palette="hls")
axs[1].fill_between(meantable['age'], meantable[yvar] - y_std,  meantable[yvar] + y_std, color='b', alpha=0.2)

axs[1].scatter(x=enigma_combat_thickness['age'], y=enigma_combat_thickness.iloc[:,i-4],s=1,color='#ba2c4a')
axs[1].set_ylim([1.8, 4])
axs[1].set_ylim([1.8, 4])
axs[1].tick_params(axis='x', labelsize=10)
axs[1].tick_params(axis='y', labelsize=10)
axs[1].legend(['ComBat-adjusted HCP mean',' +/- 1 standard deviation','ComBat-Adjusted ENIGMA'])

# axs[rowcount,counter%5].get_legend().remove()
# axs[rowcount,counter%5].legend(fontsize=8)
counter = counter +1
if (counter%5 ==0) & (counter!=0):
    rowcount = rowcount+1

plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/regions_meanstd_smooth.jpeg')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/regions_meanstd_smooth.pdf')

In [161]:

i=34
yvar= meantable.columns[i]

In [41]:
i=1

agesub = np.int16(np.round(enigma_combat_thickness.iloc[i,71])) # 71 = age
print(agesub)
meantable[meantable['age']==agesub]



67


,age,size,lh_bankssts_thickness_mean,lh_caudalanteriorcingulate_thickness_mean,lh_caudalmiddlefrontal_thickness_mean,lh_cuneus_thickness_mean,lh_entorhinal_thickness_mean,lh_fusiform_thickness_mean,lh_inferiorparietal_thickness_mean,lh_inferiortemporal_thickness_mean,...,rh_rostralmiddlefrontal_thickness_mean,rh_superiorfrontal_thickness_mean,rh_superiorparietal_thickness_mean,rh_superiortemporal_thickness_mean,rh_supramarginal_thickness_mean,rh_frontalpole_thickness_mean,rh_temporalpole_thickness_mean,rh_transversetemporal_thickness_mean,rh_insula_thickness_mean,rh_MeanThickness_thickness_mean
45,67.0,19.0,2.432845,2.238085,2.456671,1.999648,3.172718,2.782931,2.382078,2.777956,...,2.321832,2.562519,2.1055,2.645618,2.431626,2.650069,3.582188,2.432889,2.895826,2.429023


In [34]:
enigma_combat_thickness.iloc[i,71]

67.0

In [36]:
meantable

,age,size,lh_bankssts_thickness_mean,lh_caudalanteriorcingulate_thickness_mean,lh_caudalmiddlefrontal_thickness_mean,lh_cuneus_thickness_mean,lh_entorhinal_thickness_mean,lh_fusiform_thickness_mean,lh_inferiorparietal_thickness_mean,lh_inferiortemporal_thickness_mean,...,rh_rostralmiddlefrontal_thickness_mean,rh_superiorfrontal_thickness_mean,rh_superiorparietal_thickness_mean,rh_superiortemporal_thickness_mean,rh_supramarginal_thickness_mean,rh_frontalpole_thickness_mean,rh_temporalpole_thickness_mean,rh_transversetemporal_thickness_mean,rh_insula_thickness_mean,rh_MeanThickness_thickness_mean
0,22.0,50.0,2.794931,2.774843,2.876617,2.170731,3.464525,3.002507,2.729418,3.016692,...,2.677975,3.016719,2.412297,3.046544,2.796278,2.971959,3.861943,2.791405,3.274090,2.752288
1,23.0,59.0,2.770126,2.736426,2.819791,2.164970,3.384116,2.958965,2.682144,2.975150,...,2.621950,2.942553,2.349188,3.020811,2.775191,2.890763,3.716307,2.776147,3.224596,2.713099
2,24.0,46.0,2.794871,2.727646,2.837124,2.172465,3.516195,2.973983,2.681262,2.982604,...,2.630147,2.937377,2.351278,3.030284,2.786372,2.873595,3.872865,2.779575,3.207415,2.715469
3,25.0,62.0,2.705372,2.699006,2.812298,2.150528,3.413748,2.938223,2.660798,2.956189,...,2.611781,2.935615,2.360682,3.008813,2.745861,2.836115,3.773281,2.727407,3.183564,2.698040
4,26.0,82.0,2.788885,2.763515,2.822806,2.191914,3.459325,2.989528,2.684422,3.008475,...,2.626663,2.953129,2.372628,3.039457,2.781493,2.894130,3.885740,2.811640,3.214372,2.722148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,90.0,5.0,2.116721,2.077955,2.278562,1.760214,2.815604,2.517446,2.062583,2.600215,...,2.185201,2.403891,1.819656,2.403279,2.173573,2.579142,3.410370,2.176878,2.626140,2.194364
69,91.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,92.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,93.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
